<a href="https://colab.research.google.com/github/Travistyse/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module3-join-and-reshape-data/LS_DS_113_Join_and_Reshape_Data_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [52]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-10-31 00:08:47--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.176.229
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.176.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  34.7MB/s    in 6.2s    

2019-10-31 00:08:54 (31.5 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [53]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [54]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01/instacart_2017_05_01


In [55]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [56]:
##### YOUR CODE HERE #####
# I should only need the product orders dataset(s) but due to pid I'll need
# The product ID -> Product "decoder" / "transcriber" haha - so Products.
# I'll need produce and dairy
import pandas as pd
import seaborn as sns
import numpy as np 

pd.options.display.max_rows = 999
pd.options.display.max_columns = 100

# Let's look at these one by one starting with the order_products

order_products__prior = pd.read_csv('order_products__prior.csv')
print(order_products__prior.shape) # Don't need it but good practice
order_products__prior.head()



(32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [57]:
df_orders = pd.read_csv('orders.csv')
print(df_orders.shape)
df_orders.head()

(3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [58]:
df_products = pd.read_csv('products.csv')
print(df_products.shape)
df_products.head()

(49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


Perfect! (: I'll need to merge some datasets, sure, but this is everything I need. 

I need the number of times a specific product was ordered - so I need product name to figure out which product IDs I'm tracking and from there I'll also need to determine how many times each was ordered.

Product ID -> Product Name is in products

Product ID -> # of Times ordered is in Order product.. kind of. I'll have to suss it out but it's in there.

Which means I don't need df_orders - disappointing considering it's an easier variable name (: oh well.

In [59]:
# Create a new data frame
# This column is every single sale.. actually let me filter that first
#df_prods_ordered = order_products__prior["product_id"]

# I'm starting by simply finding my pid values first
# I may have to merge them and blahblahblah but I think this is more efficient
df_products["product_name"]["Banana"]

KeyError: ignored

In [60]:
df_products['product_name']

0                               Chocolate Sandwich Cookies
1                                         All-Seasons Salt
2                     Robust Golden Unsweetened Oolong Tea
3        Smart Ones Classic Favorites Mini Rigatoni Wit...
4                                Green Chile Anytime Sauce
                               ...                        
49683            Vodka, Triple Distilled, Twist of Vanilla
49684                   En Croute Roast Hazelnut Cranberry
49685                                     Artisan Baguette
49686           Smartblend Healthy Metabolism Dry Cat Food
49687                               Fresh Foaming Cleanser
Name: product_name, Length: 49688, dtype: object

In [0]:
# Just going to do what we're supposed to - just wanted to explore a bit.
# But otherwise I think that this would work w/ some Numpy but I've forgotten
# And don't want to waste time on a tangent by looking it up right now (:

# Every product that I'm looking for (their pids)
product_names = [
  'Banana', 'Bag of Organic Bananas', 'Organic Strawberries',
  'Organic Baby Spinach', 'Organic Hass Avocado', 'Organic Avocado',
  'Large Lemon', 'Strawberries', 'Limes', 'Organic Whole Milk'
]

In [0]:
#df_products.columns.to_list()
# Seems like this isn't applied and is just for observation
# RIP df_prods_ordered - merged it is
merged = (
    df_products[['product_id', 'product_name']]
    .merge(order_products__prior[['order_id', 'product_id']])
)
# Don't want to add a comment and mess that up but to sum up what's happening:
# I'm including order_id because both have product_id and while one is a 
# translation device the other is the data I'm analyzing. 
# Frankly, I still strongly believe I shouldn't be doing this without first
# or during this operation culling df_products SIGNIFICANTLY. I foresee headache

In [63]:
merged.head()

,product_id,product_name,order_id
0,1,Chocolate Sandwich Cookies,1107
1,1,Chocolate Sandwich Cookies,5319
2,1,Chocolate Sandwich Cookies,7540
3,1,Chocolate Sandwich Cookies,9228
4,1,Chocolate Sandwich Cookies,9273


In [64]:
# Is this numpy or python? Guessing python..
merged.dtype

AttributeError: ignored

In [65]:
merged['product_name'].dtype

dtype('O')

In [66]:
# I'm wanting to use product_names instead of a huge list because obvious
# So let's try this programmatically 
# df[df['A'].isin([3, 6])]
condition = (
    (merged['product_name'].isin(product_names))
)
condition

0           False
1           False
2           False
3           False
4           False
            ...  
32434484    False
32434485    False
32434486    False
32434487    False
32434488    False
Name: product_name, Length: 32434489, dtype: bool

In [67]:
'''
# So that appears to be (I hope) nicely sorted data for my orders
# Let's filter it down to my product list

condition = (
    
    (merged['product_name']=='Half Baked Frozen Yogurt') | 
    (merged['product_name']=='Half And Half Ultra Pasteurized')

    )

#merged = merged[condition]

#print(merged.shape)
#merged.head()
'''

"\n# So that appears to be (I hope) nicely sorted data for my orders\n# Let's filter it down to my product list\n\ncondition = (\n    \n    (merged['product_name']=='Half Baked Frozen Yogurt') | \n    (merged['product_name']=='Half And Half Ultra Pasteurized')\n\n    )\n\n#merged = merged[condition]\n\n#print(merged.shape)\n#merged.head()\n"

In [68]:
# So that's what I just replaced ^ doing that for 10 products
# I'm hoping this actually works without repurcussions, though D:
# I'm not trying to "out smart" anything - just really considering this
# From my mindset - like what would I do if there was no Lambda or lecture notes
# What I'm really asking is "if I had no guidance, how screwed am I atm?"

merged = merged[condition]
print(merged.shape)
merged.head()

(2323158, 3)


,product_id,product_name,order_id
7549085,13176,Bag of Organic Bananas,5
7549086,13176,Bag of Organic Bananas,27
7549087,13176,Bag of Organic Bananas,29
7549088,13176,Bag of Organic Bananas,32
7549089,13176,Bag of Organic Bananas,42


In [0]:
# Tiny dance party ensues
# But yeah, I'm challenging my assumptions and understanding
# This is a growth pattern - not an elitist / "too good" / not taking it seriously
# Or outright ignoring instruction because I think my process is best

# So now it's time to GET. THOSE. NUMBERS! :D
#merged['product_name'].sum().sort_values() #describe / info attempted
# OKAY so that basically crashed my runtime. 7 minutes and nothing so.. fuck.
# Can't hit the stop button either so.. gonna try to keep moving

In [70]:
# Let's subset this per product. I want to test this first, then write a func.
condition = (
    (merged['product_name']=='Bag of Organic Bananas')
    )
subset = merged[condition]
print(subset.shape)

(379450, 3)


In [0]:
# I don't mean to be hasty but.. isn't 379,450 the number of orders I have?
# I mean, it sounds like I can just use that instead of running sum..

# Alright, so this is going to be a loop which
# Create two lists - l_pn, l_pc (product name, purchase count, local)
# For each product_names (banana and so on)
# Run the function which will output the length
# Store the length / number of rows
# Store the product names
# Create a df with two columns - product name and purchase count - using the lists

In [71]:
len(subset)

379450

In [72]:
# So it looks like no function is needed and I was about to way overdo this

l_pn = []
l_pc = []

i = -1
for prod in product_names:
  i += 1
  l_pc = len(merged[merged['product_name']==prod])
  l_pn = prod
  print(l_pc)
  print(l_pn)

472565
Banana
379450
Bag of Organic Bananas
264683
Organic Strawberries
241921
Organic Baby Spinach
213584
Organic Hass Avocado
176815
Organic Avocado
152657
Large Lemon
142951
Strawberries
140627
Limes
137905
Organic Whole Milk


In [73]:
# I could sort them but the purpose is to get them back into a df
# For further analysis and so on
df_product_order_sum = []
df_product_order_sum['product_name'] = l_pn

TypeError: ignored

In [74]:
df_product_order_sum = pd.DataFrame()
df_product_order_sum['product_name'] = l_pn
df_product_order_sum['order_count'] = l_pc
# I think this won't work but I'll try
df_product_order_sum['order_count'].sort_values()

Series([], Name: order_count, dtype: int64)

In [75]:
df_product_order_sum.head()


,product_name,order_count


In [76]:
# Okay.. so that's weird. I can't run shape
df_product_order_sum.shape()

TypeError: ignored

In [77]:
# Because this is a tuple and not pandas.. 
# Which means I did something wrong. Yup. 
df_product_order_sum = pd.DataFrame(list(zip(l_pn, l_pc)), 
               columns = ['product_name', 'order_count']) 

TypeError: ignored

In [78]:
l_pc.dtype # So it's an int - thanks, error message c:

AttributeError: ignored

In [0]:
# Alright this is getting dumb. Just going to re-run the code but using 
# Pandas from the start.
df_product_order_sum = pd.DataFrame(columns=['product_name','num_times_ordered'])

for prod in product_names:
  df_product_order_sum = df_product_order_sum.append(
      {
          'product_name': prod
      }, ignore_index = True
  )
  # ^ Replaces the following line of code
  #df_product_order_sum['num_times_ordered'].append(prod)
  df_product_order_sum = df_product_order_sum.append(
      {
          'num_times_ordered': len(merged[merged['product_name']==prod])
      }, ignore_index = True
  )

#  df_product_order_sum['product_name'].append(
#      len(merged[merged['product_name']==prod]))

In [80]:
df_product_order_sum.head()

,product_name,num_times_ordered
0,Banana,NaN
1,NaN,472565
2,Bag of Organic Bananas,NaN
3,NaN,379450
4,Organic Strawberries,NaN


In [81]:
df_product_order_sum.shape()

TypeError: ignored

In [82]:
# *FLIPS TABLE*
# xD
# Standup meeting helped me out - changing direction (:
 merged['product_name'].value_counts() #Yep. One line. x.x sum() = mistake

Banana                    472565
Bag of Organic Bananas    379450
Organic Strawberries      264683
Organic Baby Spinach      241921
Organic Hass Avocado      213584
Organic Avocado           176815
Large Lemon               152657
Strawberries              142951
Limes                     140627
Organic Whole Milk        137905
Name: product_name, dtype: int64

## Reshape Data Section

- Replicate the lesson code
- Complete the code cells we skipped near the beginning of the notebook
- Table 2 --> Tidy
- Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [0]:
flights = sns.load_dataset('flights')

In [0]:
##### YOUR CODE HERE #####
%matplotlib inline

In [88]:
flights.head()

,year,month,passengers
0,1949,January,112
1,1949,February,118
2,1949,March,132
3,1949,April,129
4,1949,May,121


In [89]:
wide = flights.pivot_table(index='year', columns='month', values='passengers')
wide

month,January,February,March,April,May,June,July,August,September,October,November,December
year,,,,,,,,,,,,
1949,112,118,132,129,121,135,148,148,136,119,104,118
1950,115,126,141,135,125,149,170,170,158,133,114,140
1951,145,150,178,163,172,178,199,199,184,162,146,166
1952,171,180,193,181,183,218,230,242,209,191,172,194
1953,196,196,236,235,229,243,264,272,237,211,180,201
1954,204,188,235,227,234,264,302,293,259,229,203,229
1955,242,233,267,269,270,315,364,347,312,274,237,278
1956,284,277,317,313,318,374,413,405,355,306,271,306
1957,315,301,356,348,355,422,465,467,404,347,305,336


Table 2 --> Tidy

In [0]:
# Looks like I'm going to need a Table 2 to start with (:
# My favorite dish is here to serve! c; Copy pasta!
table1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11], 
     [3,      1]],
    index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
    columns=['treatmenta', 'treatmentb'])

table2 = table1.T

In [92]:
table1.head()

,treatmenta,treatmentb
John Smith,NaN,2
Jane Doe,16.0,11
Mary Johnson,3.0,1


In [91]:
table2.head()

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


In [93]:
table2 = table2.reset_index()
table2

,index,John Smith,Jane Doe,Mary Johnson
0,treatmenta,NaN,16.0,3.0
1,treatmentb,2.0,11.0,1.0


In [94]:
tidy_table2 = table2.melt(id_vars='index', value_vars=['treatmenta', 'treatmentb'])
tidy_table2

KeyError: ignored

In [96]:
tidy_table2 = table2.melt(id_vars='index', value_vars=['John Smith', 'Jane Doe', 'Mary Johnson'])
tidy_table2

,index,variable,value
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [97]:
tidy_table2 = tidy_table2.rename(columns={
    'index': 'trt', 
    'variable': 'name', 
    'value': 'result'
})
tidy_table2

,trt,name,result
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [99]:
tidy_table2.trt = tidy_table2.trt.str.replace('treatment', '')

tidy_table2

,trt,name,result
0,a,John Smith,NaN
1,b,John Smith,2.0
2,a,Jane Doe,16.0
3,b,Jane Doe,11.0
4,a,Mary Johnson,3.0
5,b,Mary Johnson,1.0


Tidy --> Table 2

In [100]:
# That.. actually took a bit of thinking and I think I'll need to take
# My own notes on the lecture notes because wow did I not grasp a few 
# things on my first few takes. Definitely wouldn't want to have to 
# come back to this on Friday without taking notes of some sort!
dirty_table2 = tidy_table2
# I'm kind of confused but I'll do my best - guessing I just kinda work in rev.
dirty_table2.head() #lmfao - yeah, messy desk head-bashing champ right here

,trt,name,result
0,a,John Smith,NaN
1,b,John Smith,2.0
2,a,Jane Doe,16.0
3,b,Jane Doe,11.0
4,a,Mary Johnson,3.0


In [101]:
# So if melt got us into this mess then pivot should get us out
wide = dirty_table2.pivot_table(index='name', columns='trt', values='result')
wide

trt,a,b
name,,
Jane Doe,16.0,11.0
John Smith,NaN,2.0
Mary Johnson,3.0,1.0


Actually.. I guess that's it!? Weird. I thought I was about to have to spend like 20 minutes finishing this.. huh. Okie doke. Here's hoping I'm not missing something.

## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [0]:
##### YOUR CODE HERE #####

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [0]:
##### YOUR CODE HERE #####